#### Exercise 1 - Kaggle Big Dataset - Reviews

### Important Libraries

In [124]:
import pandas as pd
import numpy as np
from scipy.io import arff
import seaborn as sns
import matplotlib.pyplot as plt
import openml


#sklearn
from sklearn.model_selection import train_test_split 
from sklearn import tree
from sklearn.metrics import recall_score,accuracy_score,confusion_matrix, ConfusionMatrixDisplay, precision_score, make_scorer,balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.model_selection import GridSearchCV


### Loading the data

In [125]:
# train data
reviews_train = pd.read_csv('data/reviews/amazon_review_ID.shuf.lrn.csv')

#test data

X_test = pd.read_csv('data/reviews/amazon_review_ID.shuf.tes.csv')


# test result

y_test =  pd.read_csv('data/reviews/amazon_review_ID.shuf.sol.ex.csv')

### Data Preprocessing and Exploration

In [126]:
# Number of samples and number of attributes
num_samples = reviews_train.shape[0]
num_attributes = reviews_train.shape[1] - 1 # last one is the class annotation
print("The number of samples is: ", str(num_samples))
print("The number of attributes is: ", str(num_attributes))

The number of samples is:  750
The number of attributes is:  10001


In [127]:
# checking for null values
reviews_train.isna().sum()

ID        0
V1        0
V2        0
V3        0
V4        0
         ..
V9997     0
V9998     0
V9999     0
V10000    0
Class     0
Length: 10002, dtype: int64

In [128]:
#print number of unknown values:
num_unknown = 0
for n in reviews_train.columns:
  num_unknown_per_feature = reviews_train.loc[:,n].isin(['unknown']).sum(axis=0)
  num_unknown += num_unknown_per_feature
  print(n, ':', num_unknown_per_feature)
print('total unknown:', num_unknown)



ID : 0
V1 : 0
V2 : 0
V3 : 0
V4 : 0
V5 : 0
V6 : 0
V7 : 0
V8 : 0
V9 : 0
V10 : 0
V11 : 0
V12 : 0
V13 : 0
V14 : 0
V15 : 0
V16 : 0
V17 : 0
V18 : 0
V19 : 0
V20 : 0
V21 : 0
V22 : 0
V23 : 0
V24 : 0
V25 : 0
V26 : 0
V27 : 0
V28 : 0
V29 : 0
V30 : 0
V31 : 0
V32 : 0
V33 : 0
V34 : 0
V35 : 0
V36 : 0
V37 : 0
V38 : 0
V39 : 0
V40 : 0
V41 : 0
V42 : 0
V43 : 0
V44 : 0
V45 : 0
V46 : 0
V47 : 0
V48 : 0
V49 : 0
V50 : 0
V51 : 0
V52 : 0
V53 : 0
V54 : 0
V55 : 0
V56 : 0
V57 : 0
V58 : 0
V59 : 0
V60 : 0
V61 : 0
V62 : 0
V63 : 0
V64 : 0
V65 : 0
V66 : 0
V67 : 0
V68 : 0
V69 : 0
V70 : 0
V71 : 0
V72 : 0
V73 : 0
V74 : 0
V75 : 0
V76 : 0
V77 : 0
V78 : 0
V79 : 0
V80 : 0
V81 : 0
V82 : 0
V83 : 0
V84 : 0
V85 : 0
V86 : 0
V87 : 0
V88 : 0
V89 : 0
V90 : 0
V91 : 0
V92 : 0
V93 : 0
V94 : 0
V95 : 0
V96 : 0
V97 : 0
V98 : 0
V99 : 0
V100 : 0
V101 : 0
V102 : 0
V103 : 0
V104 : 0
V105 : 0
V106 : 0
V107 : 0
V108 : 0
V109 : 0
V110 : 0
V111 : 0
V112 : 0
V113 : 0
V114 : 0
V115 : 0
V116 : 0
V117 : 0
V118 : 0
V119 : 0
V120 : 0
V121 : 0
V122 : 0
V12

In [129]:
print(reviews_train.dtypes)

ID         int64
V1         int64
V2         int64
V3         int64
V4         int64
           ...  
V9997      int64
V9998      int64
V9999      int64
V10000     int64
Class     object
Length: 10002, dtype: object


In [112]:
# Plotting the categorical variables
categorical_columns = reviews_train.columns.difference(['ID','Class'])

In [130]:
# plt.figure(figsize=(20,30))
# for i,column in enumerate(categorical_columns):
#     plt.subplot(4, 4, i+1)
#     sns.histplot(x=congress_train[column],hue =congress_train['class'],multiple = 'stack')
#     plt.xlabel(column)
#     plt.xticks(rotation=90)
# plt.suptitle("Distribution between the 2 classes ", fontsize=20)
# plt.show()

In [131]:
# List of possible classes
print("List of unique class labels:",'\n',reviews_train['Class'].unique())
print('\n')
print('Number of unique class labels','\n',reviews_train['Class'].nunique())


List of unique class labels: 
 ['Grove' 'Chell' 'Auken' 'Davisson' 'Vision' 'Harp' 'Calvinnme' 'Vernon'
 'Cutey' 'Hayes' 'Robert' 'Wilson' 'Sherwin' 'Lee' 'Mahlers2nd' 'Merritt'
 'Power' 'Lovitt' 'Peterson' 'Nigam' 'Agresti' 'Comdet' 'Blankenship'
 'Chandler' 'Walters' 'Mark' 'Neal' 'Brody' 'Mitchell' 'Taylor' 'Shea'
 'Chachra' 'Messick' 'Kolln' 'Ashbacher' 'Johnson' 'Koenig' 'Corn' 'Brown'
 'Riley' 'Engineer' 'Lawyeraau' 'Janson' 'McKee' 'Dent' 'Cholette'
 'Goonan' 'CFH' 'Bukowsky' 'Morrison']


Number of unique class labels 
 50


In [173]:
reviews_train

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V9993,V9994,V9995,V9996,V9997,V9998,V9999,V10000,Class,class
0,0,14,9,8,12,7,4,5,3,2,...,1,0,1,0,1,1,0,0,Grove,0
1,1,12,3,8,6,5,3,2,3,2,...,0,0,0,0,0,0,0,0,Chell,1
2,2,17,5,7,12,8,3,0,3,2,...,1,0,0,0,0,1,0,0,Auken,2
3,3,9,6,10,6,4,7,7,3,3,...,0,0,0,2,0,0,0,0,Davisson,3
4,4,8,4,5,5,1,7,0,0,0,...,0,3,0,0,0,0,0,0,Vision,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,745,9,4,3,4,6,7,2,1,0,...,1,1,2,0,0,0,0,0,Engineer,40
746,746,13,6,1,13,2,5,7,5,3,...,0,0,1,0,0,0,1,0,Morrison,49
747,747,19,9,12,3,9,7,5,2,5,...,0,0,0,0,0,0,1,0,CFH,47
748,748,5,14,16,7,2,12,9,3,7,...,0,0,0,0,0,0,3,0,Brown,38


In [132]:
reviews_train['class'] = reviews_train['Class'].factorize()[0]

- As seen below, there is only one unique class in y_test. Therefore, we are going to replace its value with the encoded value that this class has on train.

In [143]:
y_test['Class'].unique()

array(['Wilson'], dtype=object)

In [144]:

reviews_train[reviews_train['Class'] =='Wilson']['class'].unique()

array([11])

In [146]:
y_test['class'] = y_test['Class'].replace('Wilson', 11)

In [148]:
y_test = y_test['class']

In [149]:
y_train = reviews_train['class']
X_train = reviews_train.drop(['ID','Class','class'],axis=1)

In [155]:
X_test = X_test.drop(['ID'],axis=1)

In [167]:
rf = RandomForestClassifier(random_state=1,n_estimators = 1000)

In [168]:
rf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=1000, random_state=1)

In [169]:
def evaluate(model):
    y_train_predict = model.predict(X_train)
    y_test_predict = model.predict(X_test)

    # accuracy
    print("#"*20)
    print("Accuracy on train: ", accuracy_score(y_true = y_train, y_pred = y_train_predict))
    print("Accuracy on test: ", accuracy_score(y_test, y_test_predict))
    plt.show()

      

In [170]:
evaluate(rf)

####################
Accuracy on train:  1.0
Accuracy on test:  0.04


In [174]:
tree_1 = tree.DecisionTreeClassifier(criterion = 'entropy', random_state=1)
tree_1.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy', random_state=1)

In [175]:
evaluate(tree_1)

####################
Accuracy on train:  1.0
Accuracy on test:  0.024
